In [1]:
#1
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SchoolIteration4').getOrCreate()

# Data loading

In [2]:
#2
# Read different datasets
df1 = spark.read.csv('dataset/SEC SRC RAW DATA 2012-13.csv', inferSchema=True, header=True)
df2 = spark.read.csv('dataset/SEC SRC RAW DATA 2013-14.csv', inferSchema=True, header=True)
df3 = spark.read.csv('dataset/SEC SRC RAW DATA 2014-15.csv', inferSchema=True, header=True)
df4 = spark.read.csv('dataset/SEC SRC RAW DATA 2015-16.csv', inferSchema=True, header=True)
df5 = spark.read.csv('dataset/SEC SRC RAW DATA 2016-17.csv', inferSchema=True, header=True)

#Append different datasets
school=df1.union(df2).union(df3).union(df4).union(df5)

# Data Exploration

In [3]:
school.count()


178

In [4]:
len(school.columns)

622

In [24]:
school_summary = school.groupBy("ac_year").count().show()

+-------+-----+
|ac_year|count|
+-------+-----+
|2012-13|   35|
|2016-17|   36|
|2015-16|   36|
|2014-15|   36|
|2013-14|   35|
+-------+-----+



In [12]:
school.printSchema()

root
 |-- ac_year: string (nullable = true)
 |-- statcd: integer (nullable = true)
 |-- statname: string (nullable = true)
 |-- area_sqkm: integer (nullable = true)
 |-- tot_population: double (nullable = true)
 |-- urban_population: double (nullable = true)
 |-- grwoth_rate: double (nullable = true)
 |-- sexratio: integer (nullable = true)
 |-- sc_population: double (nullable = true)
 |-- st_population: double (nullable = true)
 |-- literacy_rate: double (nullable = true)
 |-- male_literacy_rate: double (nullable = true)
 |-- female_literacy_rate: double (nullable = true)
 |-- distcd: integer (nullable = true)
 |-- blkcd: integer (nullable = true)
 |-- villages: integer (nullable = true)
 |-- clusters: integer (nullable = true)
 |-- schools: integer (nullable = true)
 |-- sch_1: integer (nullable = true)
 |-- sch_2: integer (nullable = true)
 |-- sch_3: integer (nullable = true)
 |-- sch_4: integer (nullable = true)
 |-- sch_5: integer (nullable = true)
 |-- sch_6: integer (nullable =

In [3]:
df5.select('statcd','statname').show(300) 

+------+--------------------+
|statcd|            statname|
+------+--------------------+
|     1|   JAMMU AND KASHMIR|
|     2|    HIMACHAL PRADESH|
|     3|              PUNJAB|
|     4|          CHANDIGARH|
|     5|         UTTARAKHAND|
|     6|             HARYANA|
|     7|               DELHI|
|     8|           RAJASTHAN|
|     9|       UTTAR PRADESH|
|    10|               BIHAR|
|    11|              SIKKIM|
|    12|   ARUNACHAL PRADESH|
|    13|            NAGALAND|
|    14|             MANIPUR|
|    15|             MIZORAM|
|    16|             TRIPURA|
|    17|           MEGHALAYA|
|    18|               ASSAM|
|    19|         WEST BENGAL|
|    20|           JHARKHAND|
|    21|              ODISHA|
|    22|        CHHATTISGARH|
|    23|      MADHYA PRADESH|
|    24|             GUJARAT|
|    25|         DAMAN & DIU|
|    26|DADRA & NAGAR HAVELI|
|    27|         MAHARASHTRA|
|    28|      ANDHRA PRADESH|
|    29|           KARNATAKA|
|    30|                 GOA|
|    31|  

# Data Preparation

In [4]:
#3
#creating new colum 'school_total' for calculating total number of schools
school = school.withColumn('school_total', school['sch_1']+school['sch_2']+school['sch_3']+school['sch_4']+school['sch_5']+school['sch_6']+school['sch_7'])
school.select('school_total').show()

+------------+
|school_total|
+------------+
|        3923|
|        3343|
|        8892|
|         145|
|        3240|
|        7450|
|        1965|
|       24357|
|       22194|
|        5819|
|         195|
|         357|
|         716|
|        1053|
|         711|
|         943|
|        1136|
|        7298|
|        9773|
|        4376|
+------------+
only showing top 20 rows



In [5]:
#4
school = school.withColumn('appear_total',school['apr_b_gen_py10']+school['apr_g_gen_py10']+school['apr_b_sc_py10']+school['apr_g_sc_py10']+school['apr_b_st_py10']+school['apr_g_st_py10']+school['apr_b_obc_py10']+school['apr_g_obc_py10'])
school = school.withColumn('pass_total',school['pass_b_gen_py10']+school['pass_g_gen_py10']+school['pass_b_sc_py10']+school['pass_g_sc_py10']+school['pass_b_st_py10']+school['pass_g_st_py10']+school['pass_b_obc_py10']+school['pass_g_obc_py10'])
school = school.withColumn('pass_percent',school['pass_total']/school['appear_total']*100)
school.select('pass_percent').show()

+------------------+
|      pass_percent|
+------------------+
|57.114158779215764|
| 75.63739376770539|
| 90.54269483799986|
| 97.88510296209265|
| 76.55706479313037|
| 73.38238699938334|
| 99.37974102526937|
| 66.24036378278625|
| 85.90893886151161|
| 71.80797688819882|
|              null|
| 88.14009159034138|
| 73.81625139138625|
|  83.3807543590866|
| 79.60159362549801|
|63.960146216801995|
| 73.17316366744218|
| 69.41011310830142|
|  82.6755083851103|
| 74.12998179947193|
+------------------+
only showing top 20 rows



In [34]:
school.select('statname').show()

+--------------------+
|            statname|
+--------------------+
|JAMMU AND KASHMIR...|
|HIMACHAL PRADESH ...|
|PUNJAB           ...|
|CHANDIGARH       ...|
|UTTARAKHAND      ...|
|HARYANA          ...|
|DELHI            ...|
|RAJASTHAN        ...|
|UTTAR PRADESH    ...|
|BIHAR            ...|
|SIKKIM           ...|
|ARUNACHAL PRADESH...|
|NAGALAND         ...|
|MANIPUR          ...|
|MIZORAM          ...|
|TRIPURA          ...|
|MEGHALAYA        ...|
|ASSAM            ...|
|WEST BENGAL      ...|
|JHARKHAND        ...|
+--------------------+
only showing top 20 rows



In [6]:
#5 updated dataset
school = school.select("ac_year" , "statname" , "literacy_rate" , "male_literacy_rate" ,
                       "female_literacy_rate" , "tot_population" , "urban_population" ,
                       "sexratio" , "school_total" , "bldg_all" , "girls_toilet_all" , 
                       "boys_toilet_all" , "cwsn_toilet_all" , "water_all" ,"electric_all" , 
                       "ramps_all" , "library_all" , "librarian_all" , "bndrywall_all" ,
                       "cwsn_sch_all" , "lab_asst_all" , "hmroom_all" , "internet_all" , 
                       "ict_lab_all" ,"pground_all" , "medchk_all" , "smdc_all" , 
                       "sch_bld_com_all" , "pta_all" ,"sch_2006_all" , "pass_percent")
school.columns

['ac_year',
 'statname',
 'literacy_rate',
 'male_literacy_rate',
 'female_literacy_rate',
 'tot_population',
 'urban_population',
 'sexratio',
 'school_total',
 'bldg_all',
 'girls_toilet_all',
 'boys_toilet_all',
 'cwsn_toilet_all',
 'water_all',
 'electric_all',
 'ramps_all',
 'library_all',
 'librarian_all',
 'bndrywall_all',
 'cwsn_sch_all',
 'lab_asst_all',
 'hmroom_all',
 'internet_all',
 'ict_lab_all',
 'pground_all',
 'medchk_all',
 'smdc_all',
 'sch_bld_com_all',
 'pta_all',
 'sch_2006_all',
 'pass_percent']

In [26]:
school.select('statname', 'ac_year').show(300)

+--------------------+-------+
|            statname|ac_year|
+--------------------+-------+
|   JAMMU AND KASHMIR|2012-13|
|    HIMACHAL PRADESH|2012-13|
|              PUNJAB|2012-13|
|          CHANDIGARH|2012-13|
|         UTTARAKHAND|2012-13|
|             HARYANA|2012-13|
|               DELHI|2012-13|
|           RAJASTHAN|2012-13|
|       UTTAR PRADESH|2012-13|
|               BIHAR|2012-13|
|              SIKKIM|2012-13|
|   ARUNACHAL PRADESH|2012-13|
|            NAGALAND|2012-13|
|             MANIPUR|2012-13|
|             MIZORAM|2012-13|
|             TRIPURA|2012-13|
|           MEGHALAYA|2012-13|
|               ASSAM|2012-13|
|         WEST BENGAL|2012-13|
|           JHARKHAND|2012-13|
|              ODISHA|2012-13|
|        CHHATTISGARH|2012-13|
|      MADHYA PRADESH|2012-13|
|             GUJARAT|2012-13|
|         DAMAN & DIU|2012-13|
|DADRA & NAGAR HAVELI|2012-13|
|         MAHARASHTRA|2012-13|
|      ANDHRA PRADESH|2012-13|
|           KARNATAKA|2012-13|
|       

In [7]:
#6
import pyspark.sql.functions as F
from functools import reduce

In [8]:
#to find count of null values column-wise
school_null_count = reduce(
    lambda a, b: a.union(b),
    (
        school.agg(
            F.count(F.when(F.isnull(c), c)).alias('NULL_Count')
        ).select(F.lit(c).alias("Column_Name"), "NULL_Count")
        for c in school.columns
    )
)

In [9]:
school_null_count.show(100)

+--------------------+----------+
|         Column_Name|NULL_Count|
+--------------------+----------+
|             ac_year|         0|
|            statname|         0|
|       literacy_rate|         2|
|  male_literacy_rate|         2|
|female_literacy_rate|         2|
|      tot_population|         2|
|    urban_population|         2|
|            sexratio|         2|
|        school_total|         0|
|            bldg_all|         0|
|    girls_toilet_all|         0|
|     boys_toilet_all|        35|
|     cwsn_toilet_all|         0|
|           water_all|         0|
|        electric_all|         0|
|           ramps_all|         0|
|         library_all|         0|
|       librarian_all|         0|
|       bndrywall_all|         0|
|        cwsn_sch_all|         5|
|        lab_asst_all|         5|
|          hmroom_all|         0|
|        internet_all|         0|
|         ict_lab_all|         0|
|         pground_all|        35|
|          medchk_all|        35|
|            s

In [33]:
school.filter(school['ac_year'] == '2012-13').select('ac_year','boys_toilet_all','pground_all','medchk_all').show(100)

+-------+---------------+-----------+----------+
|ac_year|boys_toilet_all|pground_all|medchk_all|
+-------+---------------+-----------+----------+
|2012-13|           null|       null|      null|
|2012-13|           null|       null|      null|
|2012-13|           null|       null|      null|
|2012-13|           null|       null|      null|
|2012-13|           null|       null|      null|
|2012-13|           null|       null|      null|
|2012-13|           null|       null|      null|
|2012-13|           null|       null|      null|
|2012-13|           null|       null|      null|
|2012-13|           null|       null|      null|
|2012-13|           null|       null|      null|
|2012-13|           null|       null|      null|
|2012-13|           null|       null|      null|
|2012-13|           null|       null|      null|
|2012-13|           null|       null|      null|
|2012-13|           null|       null|      null|
|2012-13|           null|       null|      null|
|2012-13|           

In [10]:
school.printSchema()

root
 |-- ac_year: string (nullable = true)
 |-- statname: string (nullable = true)
 |-- literacy_rate: double (nullable = true)
 |-- male_literacy_rate: double (nullable = true)
 |-- female_literacy_rate: double (nullable = true)
 |-- tot_population: double (nullable = true)
 |-- urban_population: double (nullable = true)
 |-- sexratio: integer (nullable = true)
 |-- school_total: integer (nullable = true)
 |-- bldg_all: integer (nullable = true)
 |-- girls_toilet_all: double (nullable = true)
 |-- boys_toilet_all: string (nullable = true)
 |-- cwsn_toilet_all: integer (nullable = true)
 |-- water_all: integer (nullable = true)
 |-- electric_all: integer (nullable = true)
 |-- ramps_all: integer (nullable = true)
 |-- library_all: integer (nullable = true)
 |-- librarian_all: integer (nullable = true)
 |-- bndrywall_all: integer (nullable = true)
 |-- cwsn_sch_all: integer (nullable = true)
 |-- lab_asst_all: integer (nullable = true)
 |-- hmroom_all: integer (nullable = true)
 |-- in

In [8]:
#7 Changing datatype of numeric values (from string to int)
from pyspark.sql.types import IntegerType
school = school.withColumn("pground_all", school["pground_all"].cast(IntegerType()))
school = school.withColumn("boys_toilet_all", school["boys_toilet_all"].cast(IntegerType()))
school = school.withColumn("medchk_all", school["medchk_all"].cast(IntegerType()))
school.printSchema()

root
 |-- ac_year: string (nullable = true)
 |-- statname: string (nullable = true)
 |-- literacy_rate: double (nullable = true)
 |-- male_literacy_rate: double (nullable = true)
 |-- female_literacy_rate: double (nullable = true)
 |-- tot_population: double (nullable = true)
 |-- urban_population: double (nullable = true)
 |-- sexratio: integer (nullable = true)
 |-- school_total: integer (nullable = true)
 |-- bldg_all: integer (nullable = true)
 |-- girls_toilet_all: double (nullable = true)
 |-- boys_toilet_all: integer (nullable = true)
 |-- cwsn_toilet_all: integer (nullable = true)
 |-- water_all: integer (nullable = true)
 |-- electric_all: integer (nullable = true)
 |-- ramps_all: integer (nullable = true)
 |-- library_all: integer (nullable = true)
 |-- librarian_all: integer (nullable = true)
 |-- bndrywall_all: integer (nullable = true)
 |-- cwsn_sch_all: integer (nullable = true)
 |-- lab_asst_all: integer (nullable = true)
 |-- hmroom_all: integer (nullable = true)
 |-- i

In [9]:
#8
from pyspark.sql.functions import mean

In [9]:
#Finding the mean of column pground_all
mean_value = school.select(mean(school['pground_all'])).collect()

#filling the missing value with the mean.
mean_pground_value = mean_value[0][0]
school_na = school.na.fill(mean_pground_value, subset=['pground_all'])

In [16]:
school_na.filter(school_na['ac_year'] == '2012-13').select('ac_year','statname','pground_all').show(100)

+-------+--------------------+-----------+
|ac_year|            statname|pground_all|
+-------+--------------------+-----------+
|2012-13|   JAMMU AND KASHMIR|       5379|
|2012-13|    HIMACHAL PRADESH|       5379|
|2012-13|              PUNJAB|       5379|
|2012-13|          CHANDIGARH|       5379|
|2012-13|         UTTARAKHAND|       5379|
|2012-13|             HARYANA|       5379|
|2012-13|               DELHI|       5379|
|2012-13|           RAJASTHAN|       5379|
|2012-13|       UTTAR PRADESH|       5379|
|2012-13|               BIHAR|       5379|
|2012-13|              SIKKIM|       5379|
|2012-13|   ARUNACHAL PRADESH|       5379|
|2012-13|            NAGALAND|       5379|
|2012-13|             MANIPUR|       5379|
|2012-13|             MIZORAM|       5379|
|2012-13|             TRIPURA|       5379|
|2012-13|           MEGHALAYA|       5379|
|2012-13|               ASSAM|       5379|
|2012-13|         WEST BENGAL|       5379|
|2012-13|           JHARKHAND|       5379|
|2012-13|  

In [88]:
school.createOrReplaceTempView('school')
result=spark.sql("select ac_year, statname, pground_all from school where statname='JAMMU AND KASHMIR'")
result.show()

+-------+-----------------+-----------+
|ac_year|         statname|pground_all|
+-------+-----------------+-----------+
|2012-13|JAMMU AND KASHMIR|       null|
|2013-14|JAMMU AND KASHMIR|       3122|
|2014-15|JAMMU AND KASHMIR|       3219|
|2015-16|JAMMU AND KASHMIR|       3235|
|2016-17|JAMMU AND KASHMIR|       3280|
+-------+-----------------+-----------+



# Processing Null Values

In [10]:
#9
# Replacing null values as a mean of a group
from pyspark.sql.functions import avg, col, when
from pyspark.sql.window import Window

w = Window().partitionBy('statname')
for i in school.columns:
    school = school.withColumn(i,when(col(i).isNull(),avg(col(i)).over(w)).otherwise(col(i)))
    

In [90]:
school.createOrReplaceTempView('school')
result=spark.sql("select ac_year, statname, pground_all from school where statname='JAMMU AND KASHMIR'")
result.show()

+-------+-----------------+-----------+
|ac_year|         statname|pground_all|
+-------+-----------------+-----------+
|2012-13|JAMMU AND KASHMIR|     3214.0|
|2013-14|JAMMU AND KASHMIR|     3122.0|
|2014-15|JAMMU AND KASHMIR|     3219.0|
|2015-16|JAMMU AND KASHMIR|     3235.0|
|2016-17|JAMMU AND KASHMIR|     3280.0|
+-------+-----------------+-----------+



In [67]:
school.filter(school['ac_year'] == '2012-13').select('ac_year','statname','boys_toilet_all','pground_all','medchk_all').show(100)

+-------+--------------------+---------------+-----------+----------+
|ac_year|            statname|boys_toilet_all|pground_all|medchk_all|
+-------+--------------------+---------------+-----------+----------+
|2012-13|              SIKKIM|         214.25|     187.75|    204.25|
|2012-13|           MEGHALAYA|        1019.75|      812.0|     472.0|
|2012-13|         WEST BENGAL|         8350.0|     6466.5|    6179.0|
|2012-13|         DAMAN & DIU|           39.0|      29.25|      38.0|
|2012-13|                 GOA|          474.5|     330.25|     431.0|
|2012-13|        CHHATTISGARH|        5557.25|     4268.0|    5197.0|
|2012-13|           RAJASTHAN|       26896.75|    19702.0|  22907.75|
|2012-13|             TRIPURA|         938.25|      797.0|    597.75|
|2012-13|               DELHI|         1590.0|     1871.0|    1574.5|
|2012-13|    HIMACHAL PRADESH|        3626.75|     3511.0|    3454.0|
|2012-13|             GUJARAT|        10204.0|    9518.75|  10180.25|
|2012-13|           

In [11]:
#10 Checking null values - final
school_null_count = reduce(
    lambda a, b: a.union(b),
    (
        school.agg(
            F.count(F.when(F.isnull(c), c)).alias('NULL_Count')
        ).select(F.lit(c).alias("Column_Name"), "NULL_Count")
        for c in school.columns
    )
)
school_null_count.show(100)

+--------------------+----------+
|         Column_Name|NULL_Count|
+--------------------+----------+
|             ac_year|         0|
|            statname|         0|
|       literacy_rate|         0|
|  male_literacy_rate|         0|
|female_literacy_rate|         0|
|      tot_population|         0|
|    urban_population|         0|
|            sexratio|         0|
|        school_total|         0|
|            bldg_all|         0|
|    girls_toilet_all|         0|
|     boys_toilet_all|         0|
|     cwsn_toilet_all|         0|
|           water_all|         0|
|        electric_all|         0|
|           ramps_all|         0|
|         library_all|         0|
|       librarian_all|         0|
|       bndrywall_all|         0|
|        cwsn_sch_all|         0|
|        lab_asst_all|         0|
|          hmroom_all|         0|
|        internet_all|         0|
|         ict_lab_all|         0|
|         pground_all|         0|
|          medchk_all|         0|
|            s

# Constructing New Fields

In [11]:
#11
# Normalization
#creating new normalized columns for percentage fields 

#creating library_percent column
school = school.withColumn('library_percent',school['library_all']/school['school_total']*100)

#creating bldg_percent column
school = school.withColumn('bldg_percent',school['bldg_all']/school['school_total']*100)

#creating girls_toilet_percent column
school = school.withColumn('girls_toilet_percent',school['girls_toilet_all']/school['school_total']*100)

#creating boys_toilet_percent column
school = school.withColumn('boys_toilet_percent',school['boys_toilet_all']/school['school_total']*100)

#creating cwsn_toilet_percent column
school = school.withColumn('cwsn_toilet_percent',school['cwsn_toilet_all']/school['school_total']*100)

#creating water_percent column
school = school.withColumn('water_percent',school['water_all']/school['school_total']*100)

#creating electric_percent column
school = school.withColumn('electric_percent',school['electric_all']/school['school_total']*100)

#creating ramps_percent column
school = school.withColumn('ramps_percent',school['ramps_all']/school['school_total']*100)

#creating librarian_percent column
school = school.withColumn('librarian_percent',school['librarian_all']/school['school_total']*100)

#creating bndrywall_percent column
school = school.withColumn('bndrywall_percent',school['bndrywall_all']/school['school_total']*100)

#creating cwsn_sch_percent column
school = school.withColumn('cwsn_sch_percent',school['cwsn_sch_all']/school['school_total']*100)

#creating lab_asst_percent column
school = school.withColumn('lab_asst_percent',school['lab_asst_all']/school['school_total']*100)

#creating hmroom_percent column
school = school.withColumn('hmroom_percent',school['hmroom_all']/school['school_total']*100)

#creating internet_percent column
school = school.withColumn('internet_percent',school['internet_all']/school['school_total']*100)

#creating ict_lab_percent column
school = school.withColumn('ict_lab_percent',school['ict_lab_all']/school['school_total']*100)

#creating pground_percent column
school = school.withColumn('pground_percent',school['pground_all']/school['school_total']*100)

#creating medchk_percent column
school = school.withColumn('medchk_percent',school['medchk_all']/school['school_total']*100)

#creating smdc_percent column
school = school.withColumn('smdc_percent',school['smdc_all']/school['school_total']*100)

#creating sch_bld_com_percent column
school = school.withColumn('sch_bld_com_percent',school['sch_bld_com_all']/school['school_total']*100)

#creating pta_percent column
school = school.withColumn('pta_percent',school['pta_all']/school['school_total']*100)

#creating sch_2006_percent column
school = school.withColumn('sch_2006_percent',school['sch_2006_all']/school['school_total']*100)

#creating urban_percent column
school = school.withColumn('urban_percent',school['urban_population']/school['tot_population']*100)




In [12]:
#creating region_result column
school = school.withColumn('region_result',F.when(F.col('pass_percent')>=80,'Meets requirement').otherwise('Needs Improvement'))
#school.select('pass_percent', 'region_result').show()
#school.columns

In [20]:
school.createOrReplaceTempView('school')
result=spark.sql("select ac_year, statname, library_percent from school")
result.show()

+-------+-----------+------------------+
|ac_year|   statname|   library_percent|
+-------+-----------+------------------+
|2012-13|     SIKKIM| 66.66666666666666|
|2013-14|     SIKKIM| 71.15384615384616|
|2014-15|     SIKKIM| 83.92857142857143|
|2015-16|     SIKKIM|  94.7136563876652|
|2016-17|     SIKKIM| 93.91304347826087|
|2012-13|  MEGHALAYA|34.683098591549296|
|2013-14|  MEGHALAYA| 31.45800316957211|
|2014-15|  MEGHALAYA| 32.62861169837914|
|2015-16|  MEGHALAYA| 34.98392282958199|
|2016-17|  MEGHALAYA|35.543278084714544|
|2012-13|WEST BENGAL| 70.43896449401413|
|2013-14|WEST BENGAL| 81.14596993846465|
|2014-15|WEST BENGAL| 92.62989927196568|
|2015-16|WEST BENGAL| 93.64331959976457|
|2016-17|WEST BENGAL| 94.18862824596968|
|2012-13|DAMAN & DIU|  90.2439024390244|
|2013-14|DAMAN & DIU| 95.23809523809523|
|2014-15|DAMAN & DIU|             100.0|
|2015-16|DAMAN & DIU|             100.0|
|2016-17|DAMAN & DIU|             100.0|
+-------+-----------+------------------+
only showing top

In [97]:
school.createOrReplaceTempView('school')
result=spark.sql("select ac_year, statname, library_percent from school order by ac_year")
result.show()

+-------+--------------------+------------------+
|ac_year|            statname|   library_percent|
+-------+--------------------+------------------+
|2012-13|          PUDUCHERRY| 94.30379746835443|
|2012-13|       UTTAR PRADESH| 79.98107596647742|
|2012-13|      MADHYA PRADESH| 29.35132648366815|
|2012-13|             GUJARAT| 75.31536986577858|
|2012-13|           KARNATAKA| 92.43588991256382|
|2012-13|         LAKSHADWEEP|             100.0|
|2012-13|ANDAMAN & NICOBAR...| 97.02970297029702|
|2012-13|             TRIPURA|  32.7677624602333|
|2012-13|    HIMACHAL PRADESH| 92.55160035895902|
|2012-13|               BIHAR| 84.44749957037291|
|2012-13|          CHANDIGARH| 96.55172413793103|
|2012-13|                 GOA| 95.70815450643777|
|2012-13|             MANIPUR| 68.66096866096866|
|2012-13|         WEST BENGAL| 70.43896449401413|
|2012-13|               DELHI| 91.95928753180661|
|2012-13|              SIKKIM| 66.66666666666666|
|2012-13|        CHHATTISGARH|  86.8939268575211|


In [52]:
# Final dataframe which will be used by different models
school_final = school.select('literacy_rate' , 'male_literacy_rate' , 'female_literacy_rate' , 'urban_percent' ,
                       'sexratio','bldg_percent','girls_toilet_percent',
                       'boys_toilet_percent','cwsn_toilet_percent','water_percent',
                       'electric_percent','ramps_percent','library_percent','librarian_percent','bndrywall_percent',
                       'cwsn_sch_percent','lab_asst_percent','hmroom_percent','internet_percent',
                       'ict_lab_percent','pground_percent','medchk_percent','smdc_percent',
                       'sch_bld_com_percent','pta_percent','sch_2006_percent','pass_percent','region_result')

# Exploratory analysis of DM algorithms

# Data Mining - Linear Regression

In [74]:
#13
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('linear_regression_docs').getOrCreate()

from pyspark.ml.regression import LinearRegression

In [75]:
#14
# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [76]:
#15
school_linear = school.select('literacy_rate' , 'male_literacy_rate' , 'female_literacy_rate' , 'urban_percent' ,
                       'sexratio','bldg_percent','girls_toilet_percent',
                       'boys_toilet_percent','cwsn_toilet_percent','water_percent',
                       'electric_percent','ramps_percent','librarian_percent','bndrywall_percent',
                       'cwsn_sch_percent','lab_asst_percent','hmroom_percent','internet_percent',
                       'ict_lab_percent','pground_percent','medchk_percent','smdc_percent',
                       'sch_bld_com_percent','pta_percent','sch_2006_percent','pass_percent')

# Here we specify the input column names which will act as features
assembler = VectorAssembler(
    inputCols=['literacy_rate' , 'male_literacy_rate' , 'female_literacy_rate' , 'urban_percent' ,
                       'sexratio','bldg_percent','girls_toilet_percent',
                       'boys_toilet_percent','cwsn_toilet_percent','water_percent',
                       'electric_percent','ramps_percent','librarian_percent','bndrywall_percent',
                       'cwsn_sch_percent','lab_asst_percent','hmroom_percent','internet_percent',
                       'ict_lab_percent','pground_percent','medchk_percent','smdc_percent',
                       'sch_bld_com_percent','pta_percent','sch_2006_percent',],
    outputCol="features")

In [77]:
#16
# Now that we've created the assembler variable, let's actually transform the data.
output = assembler.transform(school_linear)

In [17]:
#17
# Using print schema, you see that the features output column has been added. 
output.printSchema()

# You can see that the features column is a dense vector that combines the various features as expected.
output.head(1)

root
 |-- literacy_rate: double (nullable = true)
 |-- male_literacy_rate: double (nullable = true)
 |-- female_literacy_rate: double (nullable = true)
 |-- urban_percent: double (nullable = true)
 |-- sexratio: double (nullable = true)
 |-- bldg_percent: double (nullable = true)
 |-- girls_toilet_percent: double (nullable = true)
 |-- boys_toilet_percent: double (nullable = true)
 |-- cwsn_toilet_percent: double (nullable = true)
 |-- water_percent: double (nullable = true)
 |-- electric_percent: double (nullable = true)
 |-- ramps_percent: double (nullable = true)
 |-- librarian_percent: double (nullable = true)
 |-- bndrywall_percent: double (nullable = true)
 |-- cwsn_sch_percent: double (nullable = true)
 |-- lab_asst_percent: double (nullable = true)
 |-- hmroom_percent: double (nullable = true)
 |-- internet_percent: double (nullable = true)
 |-- ict_lab_percent: double (nullable = true)
 |-- pground_percent: double (nullable = true)
 |-- medchk_percent: double (nullable = true)

[Row(literacy_rate=82.2, male_literacy_rate=87.29, female_literacy_rate=76.43, urban_percent=1.620892231236321, sexratio=889.0, bldg_percent=100.0, girls_toilet_percent=50.97435897435898, boys_toilet_percent=109.87179487179488, cwsn_toilet_percent=1.0256410256410255, water_percent=98.46153846153847, electric_percent=90.76923076923077, ramps_percent=2.564102564102564, librarian_percent=18.461538461538463, bndrywall_percent=45.12820512820513, cwsn_sch_percent=2.564102564102564, lab_asst_percent=50.256410256410255, hmroom_percent=92.3076923076923, internet_percent=52.307692307692314, ict_lab_percent=13.846153846153847, pground_percent=96.28205128205128, medchk_percent=104.74358974358975, smdc_percent=75.38461538461539, sch_bld_com_percent=61.02564102564103, pta_percent=65.12820512820512, sch_2006_percent=2.051282051282051, pass_percent=94.05411721316297, features=DenseVector([82.2, 87.29, 76.43, 1.6209, 889.0, 100.0, 50.9744, 109.8718, 1.0256, 98.4615, 90.7692, 2.5641, 18.4615, 45.1282, 2

In [78]:
#18
# Let's select two columns (the feature and predictor).
# This is now in the appropriate format to be processed by Spark.
final_data = output.select("features",'pass_percent')
final_data.show()

+--------------------+------------------+
|            features|      pass_percent|
+--------------------+------------------+
|[82.2,87.29,76.43...| 94.05411721316297|
|[82.2,87.29,76.43...| 92.08577428981984|
|[82.2,87.29,76.43...| 97.93665531827091|
|[82.2,87.29,76.43...| 92.13992203139817|
|[82.2,87.29,76.43...| 94.05411721316297|
|[75.48,77.17,73.7...| 73.17316366744218|
|[75.48,77.17,73.7...|  84.2525228819526|
|[75.48,77.17,73.7...| 75.49961069296653|
|[75.48,77.17,73.7...| 72.49463025467935|
|[75.48,77.17,73.7...| 99.17333024297909|
|[77.08,82.67,71.1...|  82.6755083851103|
|[77.08,82.67,71.1...| 82.28316865871777|
|[77.08,82.67,71.1...| 82.51991006784687|
|[77.08,82.67,71.1...| 82.65169063123699|
|[77.08,82.67,71.1...| 93.15251417438323|
|[87.07,91.48,79.5...| 88.37727441515038|
|[87.07,91.48,79.5...| 90.31664212076583|
|[87.07,91.48,79.5...| 64.57627118644068|
|[87.07,91.48,79.5...|60.252131998516866|
|[87.07,91.48,79.5...| 62.42496998799519|
+--------------------+------------

In [79]:
# Let's do a randomised 80/20 split. 
# Remember, you can use other splits depending on how easy/difficult it is to train your model.
train_data,test_data = final_data.randomSplit([0.8,0.2])

In [80]:
# Let's see our training data.
train_data.describe().show()

# And our testing data.
test_data.describe().show()

+-------+------------------+
|summary|      pass_percent|
+-------+------------------+
|  count|               143|
|   mean|  79.8966210480045|
| stddev|12.996232128717779|
|    min| 36.74911660777385|
|    max| 99.37974102526937|
+-------+------------------+

+-------+------------------+
|summary|      pass_percent|
+-------+------------------+
|  count|                35|
|   mean| 82.81608756327331|
| stddev|14.385403020601473|
|    min| 41.75869120654397|
|    max| 99.92422034392305|
+-------+------------------+



In [81]:
#19
#Linear Regression Model
lr = LinearRegression(labelCol='pass_percent')

In [82]:
#20
# Fit the model to the data.
lrModel = lr.fit(train_data)

In [83]:
#21
# Print the coefficients and intercept for linear regression.
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [-40.18502527240927,18.87642771482375,20.95187426656881,-0.18711301017241116,-0.12954217062983614,0.3555478413456307,-0.0006259754776726217,-0.014745641876617019,-0.021367667316828598,0.891027254395276,-0.004435406525103042,-0.09683573928886566,0.11210172193088036,0.09405492520810697,1.4859880381540296,0.014045156007860061,0.09105399068518688,0.03709788204820762,0.005274710602931511,-0.15503878316770836,0.014063017135355824,-0.018486841337294106,-0.1359939674726993,0.029894349918114148,-0.036321146036024786] Intercept: 131.17249224065102


In [84]:
#22
# Let's evaluate the model against the test data.
test_results = lrModel.evaluate(test_data)

In [85]:
#23
# Interesting results! This shows the difference between the predicted value and the test data.
test_results.residuals.show()

# Let's get some evaluation metrics (as discussed in the previous linear regression notebook).
print("RSME: {}".format(test_results.rootMeanSquaredError))

+--------------------+
|           residuals|
+--------------------+
| -3.6386504398026602|
| -10.775005745542323|
| -1.1190137206226893|
|  -5.602757695796313|
| -10.217151325473374|
|  -6.040764698631563|
| -1.2457184100624943|
| -0.5814117044396028|
|   4.792136557594787|
|  -8.557814814028532|
|   2.940624715977762|
|-0.00323389819948261|
| -3.1211340269673684|
| -1.5261891643861674|
|  2.3533874796905394|
|  2.6660417700799854|
|   9.762866951974999|
|  11.588329590396441|
|  3.5425036789345796|
| -14.611984430132864|
+--------------------+
only showing top 20 rows

RSME: 8.594431987306693


In [86]:
#23
# We can also get the R2 value. 
print("R2: {}".format(test_results.r2))

R2: 0.6325659342380789


In [87]:
final_data_linear = final_data

In [35]:
final_data.describe().show()

+-------+------------------+
|summary|      pass_percent|
+-------+------------------+
|  count|               178|
|   mean| 80.47067345269218|
| stddev|13.289412642983894|
|    min| 36.74911660777385|
|    max| 99.92422034392305|
+-------+------------------+



In [36]:
# Let's just select the features column (removing the label column).
unlabeled_data = final_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[82.2,87.29,76.43...|
|[82.2,87.29,76.43...|
|[82.2,87.29,76.43...|
|[82.2,87.29,76.43...|
|[82.2,87.29,76.43...|
|[75.48,77.17,73.7...|
|[75.48,77.17,73.7...|
|[75.48,77.17,73.7...|
|[75.48,77.17,73.7...|
|[75.48,77.17,73.7...|
|[77.08,82.67,71.1...|
|[77.08,82.67,71.1...|
|[77.08,82.67,71.1...|
|[77.08,82.67,71.1...|
|[77.08,82.67,71.1...|
|[87.07,91.48,79.5...|
|[87.07,91.48,79.5...|
|[87.07,91.48,79.5...|
|[87.07,91.48,79.5...|
|[87.07,91.48,79.5...|
+--------------------+
only showing top 20 rows



In [37]:
# Now we can transform the unlabeled data.
predictions = lrModel.transform(unlabeled_data)

In [38]:
# It worked! Feeding the unlabeled data features into the model results in a prediction, 
# which is the amount someone with those features is likely to spend in a year.
predictions.show()
predictions.head(1)

+--------------------+-----------------+
|            features|       prediction|
+--------------------+-----------------+
|[82.2,87.29,76.43...|80.51812153883591|
|[82.2,87.29,76.43...|82.55896248876275|
|[82.2,87.29,76.43...|83.13166345411778|
|[82.2,87.29,76.43...|82.04205125496767|
|[82.2,87.29,76.43...|89.08717679996707|
|[75.48,77.17,73.7...| 78.7450232492116|
|[75.48,77.17,73.7...|78.83978970889689|
|[75.48,77.17,73.7...| 77.0332563878528|
|[75.48,77.17,73.7...|78.60127168495089|
|[75.48,77.17,73.7...|83.13752034229884|
|[77.08,82.67,71.1...|83.16847613292833|
|[77.08,82.67,71.1...|85.91803864880414|
|[77.08,82.67,71.1...|  86.479098246682|
|[77.08,82.67,71.1...|86.96236828835679|
|[77.08,82.67,71.1...|89.07993209955642|
|[87.07,91.48,79.5...| 64.6953646880184|
|[87.07,91.48,79.5...|70.22276952074111|
|[87.07,91.48,79.5...|66.22933814527465|
|[87.07,91.48,79.5...|67.90429181478586|
|[87.07,91.48,79.5...|71.25891639713771|
+--------------------+-----------------+
only showing top

[Row(features=DenseVector([82.2, 87.29, 76.43, 1.6209, 889.0, 100.0, 50.9744, 109.8718, 1.0256, 98.4615, 90.7692, 2.5641, 18.4615, 45.1282, 2.5641, 50.2564, 92.3077, 52.3077, 13.8462, 96.2821, 104.7436, 75.3846, 61.0256, 65.1282, 2.0513]), prediction=80.51812153883591)]

# Exploring Classification Tree Models

In [14]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree_methods_adv').getOrCreate()

In [15]:
# load data
school_tree = school_final.select('literacy_rate' , 'male_literacy_rate' , 'female_literacy_rate' , 'urban_percent' ,
                       'sexratio','bldg_percent','girls_toilet_percent',
                       'boys_toilet_percent','cwsn_toilet_percent','water_percent',
                       'electric_percent','ramps_percent','librarian_percent','bndrywall_percent',
                       'cwsn_sch_percent','lab_asst_percent','hmroom_percent','internet_percent',
                       'ict_lab_percent','pground_percent','medchk_percent','smdc_percent',
                       'sch_bld_com_percent','pta_percent','sch_2006_percent', 'region_result')

In [16]:
# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [17]:
# Combine all features into one vector named features.
assembler = VectorAssembler(
  inputCols=['literacy_rate' , 'male_literacy_rate' , 'female_literacy_rate' , 'urban_percent' ,
                       'sexratio','bldg_percent','girls_toilet_percent',
                       'boys_toilet_percent','cwsn_toilet_percent','water_percent',
                       'electric_percent','ramps_percent','librarian_percent','bndrywall_percent',
                       'cwsn_sch_percent','lab_asst_percent','hmroom_percent','internet_percent',
                       'ict_lab_percent','pground_percent','medchk_percent','smdc_percent',
                       'sch_bld_com_percent','pta_percent','sch_2006_percent'],
              outputCol="features")

In [18]:
# Let's transform the data. 
output = assembler.transform(school_tree)

In [20]:
output.head(1)

[Row(literacy_rate=82.2, male_literacy_rate=87.29, female_literacy_rate=76.43, urban_percent=1.620892231236321, sexratio=889.0, bldg_percent=100.0, girls_toilet_percent=50.97435897435898, boys_toilet_percent=109.87179487179488, cwsn_toilet_percent=1.0256410256410255, water_percent=98.46153846153847, electric_percent=90.76923076923077, ramps_percent=2.564102564102564, librarian_percent=18.461538461538463, bndrywall_percent=45.12820512820513, cwsn_sch_percent=2.564102564102564, lab_asst_percent=50.256410256410255, hmroom_percent=92.3076923076923, internet_percent=52.307692307692314, ict_lab_percent=13.846153846153847, pground_percent=96.28205128205128, medchk_percent=104.74358974358975, smdc_percent=75.38461538461539, sch_bld_com_percent=61.02564102564103, pta_percent=65.12820512820512, sch_2006_percent=2.051282051282051, region_result='Meets requirement', features=DenseVector([82.2, 87.29, 76.43, 1.6209, 889.0, 100.0, 50.9744, 109.8718, 1.0256, 98.4615, 90.7692, 2.5641, 18.4615, 45.1282

In [21]:
school_tree.head(1)

[Row(literacy_rate=82.2, male_literacy_rate=87.29, female_literacy_rate=76.43, urban_percent=1.620892231236321, sexratio=889.0, bldg_percent=100.0, girls_toilet_percent=50.97435897435898, boys_toilet_percent=109.87179487179488, cwsn_toilet_percent=1.0256410256410255, water_percent=98.46153846153847, electric_percent=90.76923076923077, ramps_percent=2.564102564102564, librarian_percent=18.461538461538463, bndrywall_percent=45.12820512820513, cwsn_sch_percent=2.564102564102564, lab_asst_percent=50.256410256410255, hmroom_percent=92.3076923076923, internet_percent=52.307692307692314, ict_lab_percent=13.846153846153847, pground_percent=96.28205128205128, medchk_percent=104.74358974358975, smdc_percent=75.38461538461539, sch_bld_com_percent=61.02564102564103, pta_percent=65.12820512820512, sch_2006_percent=2.051282051282051, region_result='Meets requirement')]

In [19]:
# Let's import the string indexer (similar to the logistic regression exercises).
from pyspark.ml.feature import StringIndexer

In [20]:
indexer = StringIndexer(inputCol="region_result", outputCol="region_result_index")
output_fixed = indexer.fit(output).transform(output)

In [21]:
# Let's select the two columns we want. Features (which contains vectors), and the predictor.
final_data = output_fixed.select("features","region_result_index")

In [22]:
final_data.show()

+--------------------+-------------------+
|            features|region_result_index|
+--------------------+-------------------+
|[82.2,87.29,76.43...|                0.0|
|[82.2,87.29,76.43...|                0.0|
|[82.2,87.29,76.43...|                0.0|
|[82.2,87.29,76.43...|                0.0|
|[82.2,87.29,76.43...|                0.0|
|[75.48,77.17,73.7...|                1.0|
|[75.48,77.17,73.7...|                0.0|
|[75.48,77.17,73.7...|                1.0|
|[75.48,77.17,73.7...|                1.0|
|[75.48,77.17,73.7...|                0.0|
|[77.08,82.67,71.1...|                0.0|
|[77.08,82.67,71.1...|                0.0|
|[77.08,82.67,71.1...|                0.0|
|[77.08,82.67,71.1...|                0.0|
|[77.08,82.67,71.1...|                0.0|
|[87.07,91.48,79.5...|                0.0|
|[87.07,91.48,79.5...|                0.0|
|[87.07,91.48,79.5...|                1.0|
|[87.07,91.48,79.5...|                1.0|
|[87.07,91.48,79.5...|                1.0|
+----------

In [23]:
# Split the training and testing set.
train_data_tree,test_data_tree = final_data.randomSplit([0.8,0.2])

In [24]:
# Let's import the relevant classifiers. 
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier
from pyspark.ml import Pipeline

# Model Comparison

In [25]:
# Use defaults to make the comparison "fair". This simplifies the comparison process.

dtc = DecisionTreeClassifier(labelCol='region_result_index',featuresCol='features')
rfc = RandomForestClassifier(labelCol='region_result_index',featuresCol='features')
gbt = GBTClassifier(labelCol='region_result_index',featuresCol='features')

In [26]:
# Train the models (it's three models, so it might take some time).
dtc_model = dtc.fit(train_data_tree)
rfc_model = rfc.fit(train_data_tree)
gbt_model = gbt.fit(train_data_tree)

In [27]:
#compare
dtc_predictions = dtc_model.transform(test_data_tree)
rfc_predictions = rfc_model.transform(test_data_tree)
gbt_predictions = gbt_model.transform(test_data_tree)

In [29]:
# Let's start off with binary classification.
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Note that the label column isn't named label, it's named PrivateIndex in this case.
my_binary_eval = BinaryClassificationEvaluator(labelCol = 'region_result_index')

In [30]:
print("DTC")
print(my_binary_eval.evaluate(dtc_predictions))

DTC
0.8055555555555556


In [31]:
print("RFC")
print(my_binary_eval.evaluate(rfc_predictions))

RFC
0.900966183574879


In [32]:
my_binary_gbt_eval = BinaryClassificationEvaluator(labelCol='region_result_index', rawPredictionCol='prediction')
print("GBT")
print(my_binary_gbt_eval.evaluate(gbt_predictions))

GBT
0.817632850241546


In [35]:
#multi-class classification evaluator
# Let's import the evaluator.
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Select (prediction, true label) and compute test error. 
acc_evaluator = MulticlassClassificationEvaluator(labelCol="region_result_index", predictionCol="prediction", metricName="accuracy")

In [36]:
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
gbt_acc = acc_evaluator.evaluate(gbt_predictions)

In [37]:
print("Here are the results!")
print('-'*40)
print('A single decision tree has an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('-'*40)
print('A random forest ensemble has an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
print('-'*40)
print('An ensemble using GBT has an accuracy of: {0:2.2f}%'.format(gbt_acc*100))

Here are the results!
----------------------------------------
A single decision tree has an accuracy of: 78.05%
----------------------------------------
A random forest ensemble has an accuracy of: 85.37%
----------------------------------------
An ensemble using GBT has an accuracy of: 82.93%


# 7.1 Test Designs

# Test Designs for Linear Regression Model

In [92]:
# Split the training and testing set: Run 1 - 90/10 split
train_data_linear_1,test_data_linear_1 = final_data_linear.randomSplit([0.9,0.1])

In [93]:
#Linear Regression Model
lr_1 = LinearRegression(labelCol='pass_percent')

In [94]:
# Fit the model to the data.
lrModel_1 = lr_1.fit(train_data_linear_1)

In [95]:
# Let's evaluate the model against the test data.
test_results_1 = lrModel_1.evaluate(test_data_linear_1)

In [96]:
# Let's get some evaluation metrics (as discussed in the previous linear regression notebook).
print("RSME: {}".format(test_results_1.rootMeanSquaredError))

RSME: 9.519241596659064


In [97]:
#23
# We can also get the R2 value. 
print("R2: {}".format(test_results_1.r2))

R2: 0.5476408161335709


In [101]:
# Split the training and testing set Run 2 - 70/30 split
train_data_linear_2,test_data_linear_2 = final_data_linear.randomSplit([0.7,0.3])

In [102]:
#Linear Regression Model
lr_2 = LinearRegression(labelCol='pass_percent')

In [103]:
# Fit the model to the data.
lrModel_2 = lr_2.fit(train_data_linear_2)

In [104]:
# Let's evaluate the model against the test data.
test_results_2 = lrModel_2.evaluate(test_data_linear_2)

In [105]:
# Let's get some evaluation metrics (as discussed in the previous linear regression notebook).
print("RSME: {}".format(test_results_2.rootMeanSquaredError))

RSME: 10.620279099697747


In [106]:
#23
# We can also get the R2 value. 
print("R2: {}".format(test_results_2.r2))

R2: 0.45884314773597545


# Test Designs for Random Forest Model

In [49]:
# Split the training and testing set - Run 1
train_data_tree_1,test_data_tree_1 = final_data.randomSplit([0.9,0.1])

In [50]:
rfc_1 = RandomForestClassifier(labelCol='region_result_index',featuresCol='features')

In [51]:
rfc_model_1 = rfc_1.fit(train_data_tree_1)

In [52]:
rfc_predictions_1 = rfc_model_1.transform(test_data_tree_1)

In [53]:
my_binary_eval_1 = BinaryClassificationEvaluator(labelCol = 'region_result_index')

In [54]:
print(my_binary_eval_1.evaluate(rfc_predictions_1))

0.9888888888888888


In [55]:
# Select (prediction, true label) and compute test error. 
acc_evaluator_1 = MulticlassClassificationEvaluator(labelCol="region_result_index", predictionCol="prediction", metricName="accuracy")

In [56]:
rfc_acc_1 = acc_evaluator_1.evaluate(rfc_predictions_1)

In [57]:
print('A random forest model has an accuracy of: {0:2.2f}%'.format(rfc_acc_1*100))

A random forest model has an accuracy of: 94.74%


In [61]:
# Split the training and testing set - Run 2
train_data_tree_2,test_data_tree_2 = final_data.randomSplit([0.7,0.3])

In [62]:
rfc_2 = RandomForestClassifier(labelCol='region_result_index',featuresCol='features')

In [63]:
rfc_model_2 = rfc_2.fit(train_data_tree_2)

In [64]:
rfc_predictions_2 = rfc_model_2.transform(test_data_tree_2)

In [65]:
my_binary_eval_2 = BinaryClassificationEvaluator(labelCol = 'region_result_index')

In [66]:
print(my_binary_eval_2.evaluate(rfc_predictions_2))

0.8244949494949492


In [67]:
# Select (prediction, true label) and compute test error. 
acc_evaluator_2 = MulticlassClassificationEvaluator(labelCol="region_result_index", predictionCol="prediction", metricName="accuracy")

In [68]:
rfc_acc_2 = acc_evaluator_2.evaluate(rfc_predictions_2)

In [69]:
print('Random forest model with 70/30 split has an accuracy of: {0:2.2f}%'.format(rfc_acc_2*100))

Random forest model with 70/30 split has an accuracy of: 74.14%


# 7.2 Conduct data mining

# Default settings for all runs

In [16]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('linear_regression_adv').getOrCreate()

# If you're getting an error with numpy, please type 'sudo pip install numpy --user' into the EC2 console.
from pyspark.ml.regression import LinearRegression

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Goal 1 - Linear Regreassion

In [20]:
#preparing dataframe for Linear regression goal 1
school_data_linear_g1 = school_final.select('literacy_rate' , 'male_literacy_rate' , 'female_literacy_rate' , 'urban_percent' , 
                                     'sexratio' , 'pass_percent')

In [21]:
assembler_g1 = VectorAssembler(
    inputCols=['literacy_rate' , 'male_literacy_rate' , 'female_literacy_rate' , 'urban_percent' , 'sexratio'],
    outputCol="features")

In [22]:
output_g1 = assembler_g1.transform(school_data_linear_g1)
output_g1.printSchema()
output_g1.head(1)

root
 |-- literacy_rate: double (nullable = true)
 |-- male_literacy_rate: double (nullable = true)
 |-- female_literacy_rate: double (nullable = true)
 |-- urban_percent: double (nullable = true)
 |-- sexratio: double (nullable = true)
 |-- pass_percent: double (nullable = true)
 |-- features: vector (nullable = true)



[Row(literacy_rate=82.2, male_literacy_rate=87.29, female_literacy_rate=76.43, urban_percent=1.620892231236321, sexratio=889.0, pass_percent=94.05411721316297, features=DenseVector([82.2, 87.29, 76.43, 1.6209, 889.0]))]

In [23]:
#Final dataframe to be fed to the model
final_data_g1 = output_g1.select("features",'pass_percent')
final_data_g1.show()

+--------------------+------------------+
|            features|      pass_percent|
+--------------------+------------------+
|[82.2,87.29,76.43...| 94.05411721316297|
|[82.2,87.29,76.43...| 92.08577428981984|
|[82.2,87.29,76.43...| 97.93665531827091|
|[82.2,87.29,76.43...| 92.13992203139817|
|[82.2,87.29,76.43...| 94.05411721316297|
|[75.48,77.17,73.7...| 73.17316366744218|
|[75.48,77.17,73.7...|  84.2525228819526|
|[75.48,77.17,73.7...| 75.49961069296653|
|[75.48,77.17,73.7...| 72.49463025467935|
|[75.48,77.17,73.7...| 99.17333024297909|
|[77.08,82.67,71.1...|  82.6755083851103|
|[77.08,82.67,71.1...| 82.28316865871777|
|[77.08,82.67,71.1...| 82.51991006784687|
|[77.08,82.67,71.1...| 82.65169063123699|
|[77.08,82.67,71.1...| 93.15251417438323|
|[87.07,91.48,79.5...| 88.37727441515038|
|[87.07,91.48,79.5...| 90.31664212076583|
|[87.07,91.48,79.5...| 64.57627118644068|
|[87.07,91.48,79.5...|60.252131998516866|
|[87.07,91.48,79.5...| 62.42496998799519|
+--------------------+------------

In [24]:
#split data for training and testing
train_data_g1,test_data_g1 = final_data_g1.randomSplit([0.8,0.2])

In [25]:
#Linear Regression Model
lr_g1 = LinearRegression(labelCol='pass_percent')

In [26]:
# Fit the model to the data.
lrModel_g1 = lr_g1.fit(train_data_g1)

In [27]:
# coefficients and intercept for linear regression Run 1.
print("Coefficients: {} Intercept: {}".format(lrModel_g1.coefficients,lrModel_g1.intercept))

Coefficients: [-43.08750893083994,20.921660100545505,22.100737137276326,-0.04775316925922438,-0.14911949342517286] Intercept: 244.9978687689212


# Goal 2 - Linear Regression

In [53]:
#preparing dataframe for Linear regression goal 2
school_data_linear_g2 = school_final.select('bldg_percent','girls_toilet_percent',
                       'boys_toilet_percent','cwsn_toilet_percent','water_percent',
                       'electric_percent','ramps_percent','library_percent','librarian_percent','bndrywall_percent',
                       'cwsn_sch_percent','lab_asst_percent','hmroom_percent','internet_percent',
                       'ict_lab_percent','pground_percent','medchk_percent','smdc_percent',
                       'sch_bld_com_percent','pta_percent','sch_2006_percent','pass_percent')

In [54]:
assembler_g2 = VectorAssembler(
    inputCols=['bldg_percent','girls_toilet_percent',
                       'boys_toilet_percent','cwsn_toilet_percent','water_percent',
                       'electric_percent','ramps_percent','library_percent','librarian_percent','bndrywall_percent',
                       'cwsn_sch_percent','lab_asst_percent','hmroom_percent','internet_percent',
                       'ict_lab_percent','pground_percent','medchk_percent','smdc_percent',
                       'sch_bld_com_percent','pta_percent','sch_2006_percent'],
    outputCol="features")

In [55]:
output_g2 = assembler_g2.transform(school_data_linear_g2)

In [57]:
final_data_g2 = output_g2.select("features",'pass_percent')
#final_data_g2.show()

In [58]:
#split data for training and testing
train_data_g2,test_data_g2 = final_data_g2.randomSplit([0.8,0.2])

In [59]:
#Linear Regression Model
lr_g2 = LinearRegression(labelCol='pass_percent')

In [60]:
# Fit the model to the data.
lrModel_g2 = lr_g2.fit(train_data_g2)

In [61]:
# Coefficients and intercept for linear regression - RUn 2.
print("Coefficients: {} Intercept: {}".format(lrModel_g2.coefficients,lrModel_g2.intercept))

Coefficients: [0.5967200700967751,-0.023424687147040227,-0.01716572201981494,0.08673233255013675,0.1302659027048301,0.21606608097549024,-0.042238206104550105,-0.02422535894170918,0.11753846267475485,-0.031748910370120675,-0.3770493501253604,-0.02409344533037698,0.1572118495913,-0.05109745275171995,0.01152453824859951,0.026663491850712658,-0.1339709403492872,-0.02370438971158362,-0.43147572046251087,0.2021535061993394,-0.13832803194285614] Intercept: -0.16025465022064503


# Goal 3 - Random Forest Classifier (Multiple Iterations)

## Iteration 1 - Random Forest with default numTrees and train-test split 90/10

In [17]:
# Configurations for Iteration 1
#preparing dataframe for Linear regression goal 3
school_data_forest_g3 = school_final.select('literacy_rate' , 'male_literacy_rate' , 'female_literacy_rate' , 'urban_percent' ,
                       'sexratio','bldg_percent','girls_toilet_percent',
                       'boys_toilet_percent','cwsn_toilet_percent','water_percent',
                       'electric_percent','ramps_percent','librarian_percent','bndrywall_percent',
                       'cwsn_sch_percent','lab_asst_percent','hmroom_percent','internet_percent',
                       'ict_lab_percent','pground_percent','medchk_percent','smdc_percent',
                       'sch_bld_com_percent','pta_percent','sch_2006_percent', 'region_result',)

In [18]:
# Combine all features into one vector named features.
assembler_g3 = VectorAssembler(
  inputCols=['literacy_rate' , 'male_literacy_rate' , 'female_literacy_rate' , 'urban_percent' ,
                       'sexratio','bldg_percent','girls_toilet_percent',
                       'boys_toilet_percent','cwsn_toilet_percent','water_percent',
                       'electric_percent','ramps_percent','librarian_percent','bndrywall_percent',
                       'cwsn_sch_percent','lab_asst_percent','hmroom_percent','internet_percent',
                       'ict_lab_percent','pground_percent','medchk_percent','smdc_percent',
                       'sch_bld_com_percent','pta_percent','sch_2006_percent'],
              outputCol="features")

In [19]:
# transform the data. 
output_g3 = assembler_g3.transform(school_data_forest_g3)

In [20]:
# Convert the output column to index
indexer_g3 = StringIndexer(inputCol="region_result", outputCol="region_result_index")
output_fixed_g3 = indexer_g3.fit(output_g3).transform(output_g3)

In [21]:
# select the required two columns we want. 
#Features (which contains vectors), and the predictor.
final_data_g3 = output_fixed_g3.select("features","region_result_index")

In [158]:
final_data_g3.show()

+--------------------+-------------------+
|            features|region_result_index|
+--------------------+-------------------+
|[82.2,87.29,76.43...|                0.0|
|[82.2,87.29,76.43...|                0.0|
|[82.2,87.29,76.43...|                0.0|
|[82.2,87.29,76.43...|                0.0|
|[82.2,87.29,76.43...|                0.0|
|[75.48,77.17,73.7...|                1.0|
|[75.48,77.17,73.7...|                0.0|
|[75.48,77.17,73.7...|                1.0|
|[75.48,77.17,73.7...|                1.0|
|[75.48,77.17,73.7...|                0.0|
|[77.08,82.67,71.1...|                0.0|
|[77.08,82.67,71.1...|                0.0|
|[77.08,82.67,71.1...|                0.0|
|[77.08,82.67,71.1...|                0.0|
|[77.08,82.67,71.1...|                0.0|
|[87.07,91.48,79.5...|                0.0|
|[87.07,91.48,79.5...|                0.0|
|[87.07,91.48,79.5...|                1.0|
|[87.07,91.48,79.5...|                1.0|
|[87.07,91.48,79.5...|                1.0|
+----------

In [22]:
# Split the training and testing set.
train_data_forest_g3,test_data_forest_g3 = final_data_g3.randomSplit([0.9,0.1])

In [157]:
# training data.
train_data_forest_g3.describe().show()

# testing data.
test_data_forest_g3.describe().show()

+-------+-------------------+
|summary|region_result_index|
+-------+-------------------+
|  count|                158|
|   mean| 0.4240506329113924|
| stddev| 0.4957694116440524|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+

+-------+-------------------+
|summary|region_result_index|
+-------+-------------------+
|  count|                 20|
|   mean|               0.55|
| stddev| 0.5104177855340404|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



In [23]:
rfc_g3 = RandomForestClassifier(labelCol='region_result_index',featuresCol='features')

In [24]:
rfc_model_g3 = rfc_g3.fit(train_data_forest_g3)

In [25]:
rfc_predictions_g3 = rfc_model_g3.transform(test_data_forest_g3)

In [26]:
binary_eval_g3 = BinaryClassificationEvaluator(labelCol = 'region_result_index')

In [27]:
print(binary_eval_g3.evaluate(rfc_predictions_g3))

0.9375


In [28]:
acc_evaluator_g3 = MulticlassClassificationEvaluator(labelCol="region_result_index", predictionCol="prediction", metricName="accuracy")

In [29]:
rfc_acc_g3 = acc_evaluator_g3.evaluate(rfc_predictions_g3)

In [48]:
print('Random forest classifier with default settings has an accuracy of: {0:2.2f}%'.format(rfc_acc_g3*100))

Random forest classifier with default settings has an accuracy of: 80.95%


## Iteration 2 - Random Forest with numTrees=10 and train-test split=80/20

In [32]:
# Configurations for Iteration 2
rfc_g3_i2 = RandomForestClassifier(labelCol='region_result_index',featuresCol='features', numTrees=10)

In [31]:
# Split the training and testing set.
train_data_forest_g3_i2,test_data_forest_g3_i2 = final_data_g3.randomSplit([0.8,0.2])

In [33]:
rfc_model_g3_i2 = rfc_g3_i2.fit(train_data_forest_g3_i2)

In [34]:
rfc_predictions_g3_i2 = rfc_model_g3_i2.transform(test_data_forest_g3_i2)

In [35]:
binary_eval_g3_i2 = BinaryClassificationEvaluator(labelCol = 'region_result_index')

In [36]:
print(binary_eval_g3_i2.evaluate(rfc_predictions_g3_i2))

0.9444444444444444


In [37]:
acc_evaluator_g3_i2 = MulticlassClassificationEvaluator(labelCol="region_result_index", predictionCol="prediction", metricName="accuracy")

In [38]:
rfc_acc_g3_i2 = acc_evaluator_g3_i2.evaluate(rfc_predictions_g3_i2)

In [41]:
print('A random forest classifier with 10 trees has an accuracy of: {0:2.2f}%'.format(rfc_acc_g3_i2*100))

A random forest classifier with 10 trees has an accuracy of: 83.33%


## Iteration 3 - Random Forest with numTrees=20 and train-test split=80/20

In [42]:
# Configurations for Iteration 3
train_data_forest_g3_i3,test_data_forest_g3_i3 = final_data_g3.randomSplit([0.8,0.2])

rfc_g3_i3 = RandomForestClassifier(labelCol='region_result_index',featuresCol='features', numTrees=20)

rfc_model_g3_i3 = rfc_g3_i3.fit(train_data_forest_g3_i3)

rfc_predictions_g3_i3 = rfc_model_g3_i3.transform(test_data_forest_g3_i3)

binary_eval_g3_i3 = BinaryClassificationEvaluator(labelCol = 'region_result_index')

print(binary_eval_g3_i3.evaluate(rfc_predictions_g3_i3))

acc_evaluator_g3_i3 = MulticlassClassificationEvaluator(labelCol="region_result_index", predictionCol="prediction", metricName="accuracy")

rfc_acc_g3_i3 = acc_evaluator_g3_i3.evaluate(rfc_predictions_g3_i3)

print('A random forest classifier with 20 trees has an accuracy of: {0:2.2f}%'.format(rfc_acc_g3_i3*100))

0.9555555555555555
A random forest classifier with 20 trees has an accuracy of: 89.29%


## Iteration 4 - Random Forest with numTrees=30 and train-test split=80/20

In [47]:
# Configurations for Iteration 4
train_data_forest_g3_i4,test_data_forest_g3_i4 = final_data_g3.randomSplit([0.8,0.2])

rfc_g3_i4 = RandomForestClassifier(labelCol='region_result_index',featuresCol='features', numTrees=30)

rfc_model_g3_i4 = rfc_g3_i4.fit(train_data_forest_g3_i4)

rfc_predictions_g3_i4 = rfc_model_g3_i4.transform(test_data_forest_g3_i4)

binary_eval_g3_i4 = BinaryClassificationEvaluator(labelCol = 'region_result_index')

print(binary_eval_g3_i4.evaluate(rfc_predictions_g3_i4))

acc_evaluator_g3_i4 = MulticlassClassificationEvaluator(labelCol="region_result_index", predictionCol="prediction", metricName="accuracy")

rfc_acc_g3_i4 = acc_evaluator_g3_i4.evaluate(rfc_predictions_g3_i4)

print('Random forest classifier with 30 trees has an accuracy of: {0:2.2f}%'.format(rfc_acc_g3_i4*100))

0.8390092879256965
Random forest classifier with 30 trees has an accuracy of: 72.22%


In [50]:
print('Iteration 1: Model with default settings has an accuracy of: {0:2.2f}%'.format(rfc_acc_g3*100))
print('Iteration 2: Model with 10 trees has an accuracy of: {0:2.2f}%'.format(rfc_acc_g3_i2*100))
print('Iteration 3: Model with 20 trees has an accuracy of: {0:2.2f}%'.format(rfc_acc_g3_i3*100))
print('Iteration 4: Model with 30 trees  has an accuracy of: {0:2.2f}%'.format(rfc_acc_g3_i4*100))

Iteration 1: Model with default settings has an accuracy of: 80.95%
Iteration 2: Model with 10 trees has an accuracy of: 83.33%
Iteration 3: Model with 20 trees has an accuracy of: 89.29%
Iteration 4: Model with 30 trees  has an accuracy of: 72.22%
